In [105]:
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"
sys.path.append(str(src_path))

In [106]:
import praw
from psaw import PushshiftAPI

secrets_path = src_path / 'api_credentials.txt'
secrets_txt = open(secrets_path, 'r')

my_id = secrets_txt.readline().split('=')[1]
my_secret = secrets_txt.readline().split('=')[1]

secrets_txt.close()

reddit = praw.Reddit(
     client_id=my_id,
     client_secret=my_secret,
     user_agent="test_script by u/Mizule_RL"
 )

s_api = PushshiftAPI(reddit)

In [41]:
pushshift_url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&author={}&before="

In [80]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time

In [96]:
start_epoch = int(dt.datetime(2020, 11, 16).timestamp())
data_size = 10000

collected_posts = []
while len(collected_posts) < data_size:
    print(f'polling pushshift for {data_size - len(collected_posts)} more posts before {start_epoch}')
    batch = list(s_api.search_submissions(before=start_epoch, subreddit='Art', limit=1000))
    for post in batch:
        if len(post.title.split(',')) == 4:
            collected_posts.append(post)
    start_epoch = int(batch[-1].created_utc)
    
len(collected_posts)

polling pushshift for 10000 more posts before 1605506400
polling pushshift for 9942 more posts before 1605494190
polling pushshift for 9868 more posts before 1605483669
polling pushshift for 9798 more posts before 1605474400
polling pushshift for 9736 more posts before 1605467638
polling pushshift for 9675 more posts before 1605458989
polling pushshift for 9607 more posts before 1605450977
polling pushshift for 9532 more posts before 1605436347
polling pushshift for 9470 more posts before 1605418521
polling pushshift for 9400 more posts before 1605404941
polling pushshift for 9339 more posts before 1605396258
polling pushshift for 9271 more posts before 1605386693
polling pushshift for 9206 more posts before 1605378135
polling pushshift for 9136 more posts before 1605369618
polling pushshift for 9073 more posts before 1605359014
polling pushshift for 9014 more posts before 1605345430
polling pushshift for 8958 more posts before 1605331360
polling pushshift for 8913 more posts before 16

polling pushshift for 712 more posts before 1603825509
polling pushshift for 648 more posts before 1603817430
polling pushshift for 585 more posts before 1603809450
polling pushshift for 515 more posts before 1603798745
polling pushshift for 452 more posts before 1603781240
polling pushshift for 396 more posts before 1603769111
polling pushshift for 339 more posts before 1603760427
polling pushshift for 291 more posts before 1603750853
polling pushshift for 226 more posts before 1603743085
polling pushshift for 165 more posts before 1603733557
polling pushshift for 110 more posts before 1603725542
polling pushshift for 49 more posts before 1603715994
polling pushshift for 3 more posts before 1603702773


10051

In [101]:
post_dicts = [{'title': post.title, 'medium': (post.title.split(',')[2].lower()), 'url': post.url, 'id': post.id, 'unix_time': int(post.created_utc)} for post in collected_posts]

In [102]:
df = pd.DataFrame(post_dicts)

In [104]:
df[df.medium == 'me']

,title,medium,url,id,unix_time
1998,"Gray Scale,Acrylics,Me,2020",me,https://i.redd.it/tkznx816xry51.jpg,jsrfx2,1605171400
4231,"2FACE ,Digital,Me,2020",me,https://i.redd.it/w09nu9b83yx51.jpg,jq5o01,1604810387
4772,"Untitled,Digital,Me,2020",me,https://i.redd.it/dujbtqy27qx51.jpg,jpifde,1604714674
4776,"Couple, Digital,Me,2020",me,https://i.redd.it/17f46hnm4qx51.jpg,jpi7qd,1604713845
5236,"Sunflowers,Pope.art,me,2020",me,https://i.redd.it/plg5ars4u8x51.jpg,jnyw20,1604504517
5411,"Eyes,graphite pencil,me,2020",me,https://i.redd.it/7zohjhxaq5x51.jpg,jnqs3i,1604466895
6977,"Save the Humans, Acrylic with photoshop edits,...",me,https://i.redd.it/b0drcgs5pjw51.jpg,jlvkhs,1604200219
7432,"How to make paper diya,paper diya,me,2020",me,https://youtu.be/Aizvht8x9Ho,jlg8wg,1604140746
7873,"Green,digital,me,2020",me,https://i.redd.it/t4fyazusv8w51.jpg,jky3rf,1604069218
8002,"Save the Humans,Acrylic with Photoshop editing...",me,https://i.redd.it/e9bpcpri66w51.jpg,jkr6xx,1604036572


In [55]:
int(dt.datetime(2018, 12, 31).timestamp())

1546236000

In [73]:
y = 'abcd'
z = y.split(',')
z

['abcd']

In [74]:
for x in testing_psaw:
    print(x.link_flair_text)

Rule 1 - Title format
Artwork
Artwork
Rule 1
Artwork
Discussion
Artwork
Artwork
Artwork
Artwork


In [58]:
testing_psaw[-1].created_utc

1546233444.0

In [12]:
for submission in reddit.subreddit("Art").hot(limit=10):
    print(type(submission.title))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [60]:
for submission in reddit.subreddit("Art").hot(limit=10):
    response = requests.get(submission.url)

    file = open(submission.id, "wb")
    file.write(response.content)
    file.close()